## Training attention based model

In [1]:

%matplotlib inline

import sys
sys.path.append("../../")

from train_model import train
import tensorflow as tf
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 4)
rcParams['figure.dpi'] = 100
rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'
rcParams['axes.facecolor'] = '#ffffff'
rcParams['lines.linewidth'] = 2.0


In [2]:
import os
file_path = r"D:\Final_file\ASI-main\output\models\kc\asi_kc_weights.hdf5"
from tensorflow.keras import backend as K
K.clear_session()
# Check if the file exists and remove it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"Removed the file: {file_path}")
else:
    print(f"The file {file_path} does not exist")



Removed the file: D:\Final_file\ASI-main\output\models\kc\asi_kc_weights.hdf5


In [3]:
hyperparameter={
"num_nearest":60,
"sigma":2,
'type_compat_funct_eucli':'kernel_gaussiano',
"geointerpolation": 'asi',
'Num_heads':8,
"learning_rate":0.001,
"batch_size":250,
"num_neuron":60,
"num_layers":3,
"size_embedded":50,
"num_nearest_geo":30,
"num_nearest_eucli":30,
"id_dataset":'kc',
"epochs":300,
"optimier":'adam',
"validation_split":0.1,
"label":'asi_kc',
"early_stopping": False,
'scale_log':True,
"graph_label":'matrix',
}


In [4]:
spatial = train(**hyperparameter)

In [5]:
dataset,\
result,\
fit,\
embedded_train,\
embedded_test,\
predict_regression_train,\
predict_regression_test = spatial()

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Epoch 1/300
63/63 [==============================] - 1s 5ms/step - loss: 4.4751 - root_mean_squared_error: 6.3486 - val_loss: 0.7475 - val_root_mean_squared_error: 1.0950

Epoch 00001: val_loss improved from inf to 0.74746, saving model to d:\Final_file\ASI-main/output/models/kc\asi_kc_weights.hdf5
Epoch 2/300
63/63 [==============================] - 0s 3ms/step - loss: 0.4308 - root_mean_squared_error: 0.6307 - val_loss: 0.2901 - val_root_mean_squared_error: 0.4021

Epoch 00002: val_loss improved from 0.74746 to 0.29006, saving model to d:\Final_file\ASI-main/output/models/kc\asi_kc_weights.hdf5
Epoch 3/300
63/63 [==============================] - 0s 3ms/step - loss: 0.1964 - root_mean_squared_error: 0.2704 - val_loss: 0.1655 - val_root_mean_squared_error: 0.2251

Epoch 00003: val_loss improved from 0.29006 to 0.16553

In [6]:
#asi
print('################# Test ##########################')
print('MALE test:.... {}'.format(result[0]))
print('RMSE test:.... {}'.format(result[1]))
print('MAPE test:.... {}'.format(result[2]))
print('################# Train ##########################')
print('MALE train:.... {}'.format(result[3]))
print('RMSE train:.... {}'.format(result[4]))
print('MAPE train:.... {}'.format(result[5]))

################# Test ##########################
MALE test:.... 0.11290006349671419
RMSE test:.... 124557.59577306532
MAPE test:.... 11.50800411449745
################# Train ##########################
MALE train:.... 0.11290006349671419
RMSE train:.... 95664.61487961246
MAPE train:.... 10.474790295084524


In [6]:
#asi_multi
print('################# Test ##########################')
print('MALE test:.... {}'.format(result[0]))
print('RMSE test:.... {}'.format(result[1]))
print('MAPE test:.... {}'.format(result[2]))
print('################# Train ##########################')
print('MALE train:.... {}'.format(result[3]))
print('RMSE train:.... {}'.format(result[4]))
print('MAPE train:.... {}'.format(result[5]))

################# Test ##########################
MALE test:.... 0.11079845133996201
RMSE test:.... 107993.67413139479
MAPE test:.... 11.374641570917305
################# Train ##########################
MALE train:.... 59496.69008488547
RMSE train:.... 94584.08319357428
MAPE train:.... 10.479219777064092


In [18]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Base models benchmarking 

In [19]:
path = r'D:\Final_file\ASI-main\datasets\kc\data.npz'
data = np.load(path)

In [20]:
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [21]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:

# Define the mean absolute percentage error
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}

# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [1, 10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [1000], 'learning_rate': [0.01, 0.05, 0.1]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0, n_estimators=1000),
        'params': {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.05]}
    },
    'XGBoost': {
        'model': XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.05]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score(np.exp(y_test), np.exp(y_pred_test))
        test_mape = mean_absolute_percentage_error(np.exp(y_test), np.exp(y_pred_test))

        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)

    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])

    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")
    print("\n")


Test Set Evaluation for Linear Regression
Average Test MAE: 0.19253815552964032, Best Test MAE: 0.19240816401081715
Average Test MSE: 43824813408.90776, Best Test MSE: 42213802813.51948
Average Test RMSE: 209329.72692923745, Best Test RMSE: 205459.97861753876
Average Test R2: 0.6788315125176732, Best Test R2: 0.6906377427327615
Average Test MAPE: 19.793398821765017, Best Test MAPE: 19.774920624415795


Test Set Evaluation for KNN
Average Test MAE: 0.15132194330901652, Best Test MAE: 0.15006840205486138
Average Test MSE: 30846631234.30832, Best Test MSE: 30291981699.081963
Average Test RMSE: 175628.22320489926, Best Test RMSE: 174045.9183637524
Average Test R2: 0.7739416297107559, Best Test R2: 0.7780063578511662
Average Test MAPE: 15.293094596840081, Best Test MAPE: 15.142555869630511


Test Set Evaluation for Decision Tree
Average Test MAE: 0.1607652763291973, Best Test MAE: 0.1582723959391721
Average Test MSE: 31853210168.653576, Best Test MSE: 25260943322.531265
Average Test RMSE: 1

## Testing the embeddings

In [13]:
X_train ,X_test, y_train , y_test = embedded_train,embedded_test,dataset.y_train,dataset.y_test

In [17]:


# Initialize lists to store test set evaluation metrics for each model
test_mae_list = {}
test_mse_list = {}
test_rmse_list = {}
test_r2_list = {}
test_mape_list = {}

# Define the models and their respective parameter grids
models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'n_neighbors': [5, 10, 15, 20]}
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor(),
        'params': {'max_depth': [5, 9, 12, 15]}
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {'n_estimators': [50, 100, 150], 'max_depth': [8, 12, 16]}
    },
    'SVM': {
        'model': SVR(),
        'params': {'C': [10, 100], 'gamma': ['scale', 'auto']}
    },
    'LightGBM': {
        'model': LGBMRegressor(),
        'params': {'n_estimators': [200,400,1000], 'learning_rate': [0.05]}
    },
    'CatBoost': {
        'model': CatBoostRegressor(verbose=0),
        'params': {'depth': [ 8, 10], 'learning_rate': [0.05],'n_estimators': [200,400,1000]}
    },
    'XGBoost': {
        'model': XGBRegressor( learning_rate=0.05, random_state=42),
        'params': {'max_depth': [5, 7, 9], 'learning_rate': [0.05],'n_estimators': [200,400,1000]}
    }
}

# Define the K-fold cross-validator
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# For each model
for name, model_info in models.items():
    test_mae_list[name] = []
    test_mse_list[name] = []
    test_rmse_list[name] = []
    test_r2_list[name] = []
    test_mape_list[name] = []


    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Grid Search for hyperparameter tuning
        grid = GridSearchCV(estimator=model_info['model'], param_grid=model_info['params'], cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
        grid_result = grid.fit(X_train_fold, y_train_fold)
        best_model = grid_result.best_estimator_

        # Make predictions on the test set
        y_pred_test = best_model.predict(X_test)

        # Evaluate the model on the test set
        test_mae = mean_absolute_error((y_test), (y_pred_test))
        test_mse = mean_squared_error(np.exp(y_test), np.exp(y_pred_test))
        test_rmse = np.sqrt(test_mse)
        test_r2 = r2_score(np.exp(y_test), np.exp(y_pred_test))
        test_mape = mean_absolute_percentage_error(np.exp(y_test), np.exp(y_pred_test))


        # Store the test set metrics in the lists
        test_mae_list[name].append(test_mae)
        test_mse_list[name].append(test_mse)
        test_rmse_list[name].append(test_rmse)
        test_r2_list[name].append(test_r2)
        test_mape_list[name].append(test_mape)


    # Calculate the average and best metrics for the test set
    avg_test_mae = np.mean(test_mae_list[name])
    avg_test_mse = np.mean(test_mse_list[name])
    avg_test_rmse = np.mean(test_rmse_list[name])
    avg_test_r2 = np.mean(test_r2_list[name])
    avg_test_mape = np.mean(test_mape_list[name])


    best_test_mae = np.min(test_mae_list[name])
    best_test_mse = np.min(test_mse_list[name])
    best_test_rmse = np.min(test_rmse_list[name])
    best_test_r2 = np.max(test_r2_list[name])
    best_test_mape = np.min(test_mape_list[name])
 

    print(f"Test Set Evaluation for {name}")
    print(f"Average Test MAE: {avg_test_mae}, Best Test MAE: {best_test_mae}")
    print(f"Average Test MSE: {avg_test_mse}, Best Test MSE: {best_test_mse}")
    print(f"Average Test RMSE: {avg_test_rmse}, Best Test RMSE: {best_test_rmse}")
    print(f"Average Test R2: {avg_test_r2}, Best Test R2: {best_test_r2}")
    print(f"Average Test MAPE: {avg_test_mape}, Best Test MAPE: {best_test_mape}")

    print("\n")


Test Set Evaluation for Linear Regression
Average Test MAE: 0.11039634696858898, Best Test MAE: 0.11031362387643998
Average Test MSE: 11477452475.345476, Best Test MSE: 11439173690.893269
Average Test RMSE: 107132.80478080029, Best Test RMSE: 106954.0728111523
Average Test R2: 0.915887923645837, Best Test R2: 0.9161684482698775
Average Test MAPE: 11.324417437721987, Best Test MAPE: 11.315195373352148


Test Set Evaluation for KNN
Average Test MAE: 0.12109989208415721, Best Test MAE: 0.12083496824494343
Average Test MSE: 15629430004.541416, Best Test MSE: 15186951890.788687
Average Test RMSE: 125010.12442712826, Best Test RMSE: 123235.35162764249
Average Test R2: 0.8854603133632699, Best Test R2: 0.8887029974840678
Average Test MAPE: 12.29238237809716, Best Test MAPE: 12.27523759697919


Test Set Evaluation for Decision Tree
Average Test MAE: 0.14122703157471622, Best Test MAE: 0.13525401458044037
Average Test MSE: 23944258843.297077, Best Test MSE: 20372190185.579433
Average Test RMSE:

c:\Users\zabde\anaconda3\envs\MODEL\lib\site-packages\sklearn\model_selection\_search.py:925: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  category=UserWarning


Test Set Evaluation for CatBoost
Average Test MAE: 0.11355875414075749, Best Test MAE: 0.11302119372564326
Average Test MSE: 12814559019.12153, Best Test MSE: 12205946695.868233
Average Test RMSE: 113174.4610619107, Best Test RMSE: 110480.52631965614
Average Test R2: 0.9060889889131225, Best Test R2: 0.9105491813045552
Average Test MAPE: 11.64407214689413, Best Test MAPE: 11.59097557681722


Test Set Evaluation for XGBoost
Average Test MAE: 0.11517638618174449, Best Test MAE: 0.11467625680322098
Average Test MSE: 12625956340.239614, Best Test MSE: 11803525139.895697
Average Test RMSE: 112332.34108247883, Best Test RMSE: 108644.02947192127
Average Test R2: 0.9074711565117941, Best Test R2: 0.9134983124567192
Average Test MAPE: 11.81387972544207, Best Test MAPE: 11.76713822433953


